In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import ipywidgets as widgets
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout, Conv2D, MaxPool2D, Flatten, Reshape, GaussianNoise
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.random import set_seed
from numpy.random import seed
from plotly.offline import init_notebook_mode
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('../input/volcanoesvenus/volcanoes_train/train_images.csv', header = None)
df

In [ ]:
y = pd.read_csv('../input/volcanoesvenus/volcanoes_train/train_labels.csv')
y

In [ ]:
y[y['Volcano?'] == 1]

In [ ]:
y['Volcano?'].value_counts()

In [ ]:
volcanos_index = y[y['Volcano?'] == 1].head().index
plt.title('Images with volcanos')
for i in volcanos_index:
    plt.imshow(df.iloc[i].values.reshape(110, 110, 1))
    plt.show()

In [ ]:
volcanos_index = y[y['Volcano?'] == 0].head().index
plt.title('Images without volcanos')
for i in volcanos_index:
    plt.imshow(df.iloc[i].values.reshape(110, 110, 1))
    plt.show()

In [ ]:
df_d = df.values.reshape(len(df), 110, 110, 1)
df_d = df_d / 255
print(f'Shape of each photo: {df_d[0].shape}')
print(f'Scaled data: \nMinimal value: {np.min(np.min(df_d))} \nMaximum value: {np.max(np.max(df_d))}')

# Modeling

### Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_d, 
                                                    y['Volcano?'], 
                                                    test_size=0.3,
                                                   random_state = 101)

### Modeling on raw data

In [ ]:
def ANN_model_evaluation(model, X_test, y_test, show_history = True):
    
    if show_history:
        histo = pd.DataFrame(model.history.history)
        for metric in [['acc', 'val_acc'], ['loss', 'val_loss']]:
            histo[metric].plot()
            plt.title(' and '.join(metric))
            plt.show()
    
    pred = model.predict_classes(X_test)
    print(classification_report(y_test, pred, target_names = ['No Volcanos', 'Volcanos']))
    display(pd.DataFrame(confusion_matrix(y_test, pred), 
                         columns = ['Predicted No Volcanos', 'Predicted Volcanos'],
                         index = ['No Volcanos', 'Volcanos']))
    print(f'Accuracy: {round(accuracy_score(y_test, pred), 5)}')

In [ ]:
es = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
seed(101)
set_seed(101)
    
model3 = Sequential([
    
    Conv2D(filters=16, kernel_size=(4,4), input_shape=(110, 110, 1), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=32, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=64, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=96, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Flatten(),
    
    Dense(64, activation='relu'),
    Dropout(0.15),
    Dense(1)
])

model3.compile(
    optimizer='adam', 
    loss='binary_crossentropy',
    metrics = ['acc']
)

print(model3.summary())

model3.fit(
    x = X_train,
    y = y_train,
    epochs = 49,
    validation_data=(X_test, y_test), 
    batch_size = 128
    #callbacks=[es]
         )

ANN_model_evaluation(model3, X_test, y_test)

### Modeling on augmented data

In [ ]:
data_augmentation = Sequential([
  RandomFlip("horizontal_and_vertical", seed = 1),
])

image = tf.expand_dims(X_train[0], 0)

plt.figure(figsize=(10, 10))
for i in range(9):
    augmented_image = data_augmentation(image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0])
    plt.axis("off")

In [ ]:
seed(101)
set_seed(101)
    
model_aug2 = Sequential([
    RandomFlip("horizontal_and_vertical", seed = 1),
    Conv2D(filters=16, kernel_size=(4,4), input_shape=(110, 110, 1), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=32, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=64, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=96, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Flatten(),
    
    Dense(64, activation='relu'),
    Dropout(0.15),
    Dense(1)
])


model_aug2.compile(
    optimizer='adam', 
    loss='binary_crossentropy',
    metrics = ['acc']
)


model_aug2.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    validation_data=(X_test, y_test), 
    batch_size = 128
    #callbacks=[es]
         )

print(model_aug2.summary())

ANN_model_evaluation(model_aug2, X_test, y_test)

In [ ]:
seed(101)
set_seed(101)
    
model_aug3 = Sequential([
    RandomFlip("horizontal_and_vertical", seed = 1),
    Conv2D(filters=16, kernel_size=(4,4), input_shape=(110, 110, 1), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=32, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=64, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(filters=96, kernel_size=(4,4), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Flatten(),
    
    Dense(64, activation='relu'),
    Dropout(0.15),
    Dense(1)
])


model_aug3.compile(
    optimizer='adam', 
    loss='binary_crossentropy',
    metrics = ['acc']
)


model_aug3.fit(
    x = X_train,
    y = y_train,
    epochs = 40,
    validation_data=(X_test, y_test), 
    batch_size = 128
    #callbacks=[es]
         )

print(model_aug3.summary())

ANN_model_evaluation(model_aug3, X_test, y_test)

# Validation

In [ ]:
model_vl = model_aug3

In [ ]:
df_validation = pd.read_csv('../input/volcanoesvenus/Volcanoes_test/test_images.csv', header = None)
y_val = pd.read_csv('../input/volcanoesvenus/volcanoes_test/test_labels.csv')['Volcano?'].values

In [ ]:
df_validation_d = df_validation.values.reshape(len(df_validation), 110, 110, 1)
df_validation_d = df_validation_d / 255
print(f'Shape of each photo: {df_validation_d[0].shape}')
print(f'Scaled data: \nMinimal value: {np.min(np.min(df_validation_d))} \nMaximum value: {np.max(np.max(df_validation_d))}')

In [ ]:
df_validation_d.shape

In [ ]:
y_val.shape

In [ ]:
ANN_model_evaluation(model_vl, df_validation_d, y_val, show_history = False)